In [1]:
import timeit
import numpy as np
import matplotlib.pyplot as plt
import pickle
from time import time

In [2]:
def init_kernel_bias(num_inp_channels, kernel_size, num_kernels,mean=0,std=0.01):
    shape = [num_inp_channels, kernel_size, kernel_size, num_kernels]
    weights = std*np.random.randn(*shape) + mean
    # weights/=np.sqrt(num_inp_channels)
    bias = std*np.random.randn(1,num_kernels) + mean
    return weights.astype(np.float32), bias.astype(np.float32)

In [3]:
w0,b0=init_kernel_bias(num_inp_channels=32,kernel_size=3,num_kernels=64)

In [5]:
inp=np.random.randn(128,64,64,32).astype(np.float32)

In [5]:
def conv2d(self,inp,kernels,biases,stride=[1,1],padding=0):     #padding=(ksz-1)/2 for same shape in stride 1
    #inp[batches,row,col,d],kernels(d,ksz,ksz,num_ker),biases[1,num_ker],stride[row,col]
    inp=inp.transpose(0,3,1,2)  #inp[batches,d,row,col]
    output=[]
    ksz=kernels.shape[1]
    num_ker=kernels.shape[3]
    if not padding:                         #take care of padding in backprop too
        padding=(ksz-1)//2                  #currently don't give 'even' ksz
    out_row,out_col=((inp.shape[2]-ksz+2*padding)//stride[0]+1),((inp.shape[3]-ksz+2*padding)//stride[1]+1)
    batches,d,row,col=inp.shape
    row+=2*padding
    col+=2*padding
    padded=np.zeros((batches,d,row,col))
    padded[:,:,padding:-padding,padding:-padding]=inp
    # Take all windows into a matrix       # TO-DO: Cache up these calculations
    window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
    slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
    ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
    kern = kernels.reshape(-1,num_ker)
    for img in padded:      #img[d,row,col]     #TODO: MAKE THIS FOR LOOP INTO SINGLE NUMPY OPERATION
        # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
        out=(np.dot(np.take(img, ind), kern))
        out=(out+biases).reshape(out_row,out_col,num_ker)
        output.append(out)
#     output=np.array([(np.dot(np.take(i,ind),kern)+biases) for i in padded]).reshape(batches,out_row,out_col,num_ker)
#     bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()		#for batches
#     output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)+biases).reshape(batches,out_row,out_col,num_ker)
    return np.array(output) #output[batches,out_row,out_col,num_ker]

In [ ]:
%timeit conv0=conv2d(1,inp,w0,b0)

In [ ]:
%timeit conv0=nn.conv2d(inp,w0,b0)

In [ ]:
conv0=conv2d(1,inp,w0,b0)

In [ ]:
%timeit d_X_inp,d_w0,d_b0=nn.conv2d_back(errors=conv0,inp=inp,kernels=w0,biases=b0)

In [ ]:
conv0.shape

In [4]:
inp=np.random.randint(0,9,(1,4,4,1)).astype(np.float32)
w0,b0=init_kernel_bias(num_inp_channels=1,kernel_size=3,num_kernels=1)
w0.shape

(1, 3, 3, 1)

In [14]:
inp=np.random.randint(0,9,(128,32,32,64)).astype(np.float32)
w0,b0=init_kernel_bias(num_inp_channels=1,kernel_size=16,num_kernels=32)
w0.shape

(1, 16, 16, 32)

In [5]:
dilation=[1,1]

In [6]:
#inp[batches,row,col,d],w0(d,ksz,ksz,num_ker),b0[1,num_ker],stride[row,col]
padding=1
stride=[2,2]
ipp=inp.transpose(0,3,1,2)  #ipp[batches,d,row,col]
output=[]
ksz=w0.shape[1]
num_ker=w0.shape[3]
if not padding: #take care of padding in backprop too
    padding=(ksz-1)//2  #currently don't give 'even' ksz
out_row,out_col=((ipp.shape[2]-ksz+2*padding-(ksz-1)*(dilation[0]-1))//stride[0]+1),((ipp.shape[3]-ksz+2*padding-(ksz-1)*(dilation[1]-1))//stride[1]+1)
batches,d,row,col=ipp.shape
row+=2*padding
col+=2*padding
padded=np.zeros((batches,d,row,col),dtype=np.float32)
padded[:,:,padding:-padding,padding:-padding]=ipp

In [7]:
%timeit padded[:,:,padding:-padding:1,padding:-padding:1]=ipp

1.55 µs ± 86.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
img=padded[0]
row,col,d

(6, 6, 1)

In [9]:
out_row=2
out_col=2

In [10]:
dksz=ksz+(ksz-1)*(dilation[0]-1)

In [11]:
# %%timeit
window=(np.arange(dksz,step=dilation[0])[:,None]*row+np.arange(dksz,step=dilation[1])).ravel()+np.arange(d)[:,None]*row*col
slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
# bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()
kern = w0.reshape(-1,num_ker)
# output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

In [25]:
img.shape,row,col,out_row,out_col,ksz

((1, 6, 6), 6, 6, 2, 2, 3)

In [26]:
window

array([[ 0,  1,  2,  6,  7,  8, 12, 13, 14]])

In [27]:
slider

array([[ 0,  1,  2,  3],
       [ 6,  7,  8,  9],
       [12, 13, 14, 15],
       [18, 19, 20, 21]])

In [28]:
ind

array([[ 0,  1,  2,  6,  7,  8, 12, 13, 14],
       [ 2,  3,  4,  8,  9, 10, 14, 15, 16],
       [12, 13, 14, 18, 19, 20, 24, 25, 26],
       [14, 15, 16, 20, 21, 22, 26, 27, 28]])

In [147]:
ind.size*batches,ind.shape

(144, (16, 9))

In [65]:
# padded=np.arange(padded.size,dtype=np.float32).reshape(padded.shape)

In [29]:
padded.shape

(1, 1, 6, 6)

In [4]:
from nnet.layers import ctake,conv2d,conv2dtranspose
from nnet import layers
from ctypes import CDLL,c_int,c_void_p

Seed: 987
Seed: 947


In [9]:
import numpy as np
from numpy import asarray
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D

In [10]:
model = Sequential()
model.add(Conv2D(1,kernel_size=3,strides=[2,2],bias_initializer='zero',padding='same',input_shape=(4, 4, 1)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 2, 2, 1)           10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [11]:
X = np.arange(16).reshape(1,4,4,1).astype(np.float32)
weights = np.ones((1,3,3,1)).astype(np.float32)

In [12]:
model.set_weights([weights.reshape(3,3,1,1),np.asarray([0])])

In [13]:
X.squeeze()

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.],
       [12., 13., 14., 15.]], dtype=float32)

In [14]:
a=conv2d(1,kernel_size=3,kernels=weights,biases=0,stride=[2,2],input_shape=(4,4,1))

In [27]:
a.d_inp.dlate

[2, 2]

In [23]:
a.ind

array([[ 7,  8,  9, 13, 14, 15, 19, 20, 21],
       [ 9, 10, 11, 15, 16, 17, 21, 22, 23],
       [19, 20, 21, 25, 26, 27, 31, 32, 33],
       [21, 22, 23, 27, 28, 29, 33, 34, 35]])

In [19]:
a.forward(X).squeeze()

array([[45., 39.],
       [66., 50.]], dtype=float32)

In [18]:
model.predict(X).squeeze()

array([[45., 39.],
       [66., 50.]], dtype=float32)

In [ ]:
coled=np.empty(ind.size*batches,dtype=np.float32).reshape(-1,d*ksz*ksz)

In [69]:
batches,out_row,out_col

(2, 3, 3)

In [70]:
coled.shape

(18, 4)

In [71]:
%%time
ctake.take(c_void_p(padded.ctypes.data),c_void_p(ind.ctypes.data),c_void_p(coled.ctypes.data),c_int(1),c_int(padded[0].size),c_int(ind.size),c_int(4))

CPU times: user 1.03 ms, sys: 138 µs, total: 1.16 ms
Wall time: 2.07 ms


0

In [72]:
a=conv2d(32,kernel_size=3,stride=[2,2],input_shape=(32,32,3))

In [55]:
a.shape

(None, 4, 4, 1)

In [74]:
## Find a way to throw away useless
layers.COLT.objs

{<nnet.layers.conv2d at 0x7fb896206710>,
 <nnet.layers.conv2d at 0x7fb8c20885d0>}

In [75]:
# coled.reshape(-1,3,3)[:,1::2]=0
# coled.reshape(-1,3,3)[:,:,1::2]=0
coled.shape

(18, 4)

In [76]:
%%time
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(img.take(ind), kern)+b0
ans=output.reshape(batches,out_row,out_col,num_ker)

CPU times: user 386 µs, sys: 0 ns, total: 386 µs
Wall time: 309 µs


In [77]:
output.shape,ans.shape

((2, 9, 1), (2, 3, 3, 1))

In [25]:
%%time
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=img.take(ind).dot(kern)
if b0 is not 0:
    output+=b0
ans=output.reshape(batches,out_row,out_col,num_ker)

CPU times: user 106 µs, sys: 0 ns, total: 106 µs
Wall time: 80.3 µs


In [28]:
%%timeit
output=np.empty((batches,out_row*out_col,num_ker))
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    output[i]=np.dot(img.take(ind), kern)
output+=b0
ans2=output.reshape(batches,out_row,out_col,num_ker)

1.24 s ± 11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [47]:
from multiprocessing import Pool

In [8]:
import concurrent

In [9]:
def convolve(padded):
    batches=len(padded)
    output=np.empty((batches,out_row*out_col,num_ker))
    for i,img in enumerate(padded):      #img[d,row,col]
        # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
        output[i]=np.dot(img.take(ind), kern)
    output+=b0
    ans=output.reshape(batches,out_row,out_col,num_ker)
    return ans

In [37]:
%%timeit
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    future_shifting = {executor.submit(convolve, padd):
                         padd for padd in [padded,padded]}

4.31 s ± 80.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
ans2.shape

(128, 64, 64, 128)

In [28]:
for future in future_shifting:
    print(future.result().shape)

(64, 64, 64, 128)
(64, 64, 64, 128)


In [14]:
# %%timeit
cut=len(padded)//2
ans=convolve(padded[:cut])

In [ ]:
Pool().starmap(convolve, [[padded[:cut]]])

In [98]:
%timeit np.array([(np.dot(np.take(i,ind),kern)) for i in padded]).reshape(batches,out_row,out_col,num_ker)

1.39 s ± 80.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [95]:
%timeit (np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

928 ms ± 54.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [96]:
%timeit (np.dot(padded.take(bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

931 ms ± 20.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
np.take(img,ind).shape,kern.shape

((4096, 288), (288, 128))

In [24]:
(np.dot(np.take(img,ind), kern)).shape

(4096, 128)

In [26]:
ans=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)+b0).reshape(batches,out_row,out_col,num_ker)

In [27]:
ans.shape

(128, 64, 64, 128)

In [28]:
np.array([(np.dot(np.take(i,ind),kern)+b0) for i in padded]).reshape(batches,out_row,out_col,num_ker).__array_interface__

{'data': (139801886789648, False),
 'strides': None,
 'descr': [('', '<f8')],
 'typestr': '<f8',
 'shape': (128, 64, 64, 128),
 'version': 3}

In [29]:
np.array([(np.dot(np.take(i,ind),kern)+b0) for i in padded]).__array_interface__

{'data': (139801886789648, False),
 'strides': None,
 'descr': [('', '<f8')],
 'typestr': '<f8',
 'shape': (128, 4096, 128),
 'version': 3}

In [4]:
N, n_N, M, n_M = 1024, 2, 16, 3
# DEBUG 
# N, n_N, M, n_M = 64, 10, 32, 8

def get_data(N=N, n_N=n_N, M=M, n_M=n_M, seed=42, prefetching=False):
    np.random.seed(seed)
    if prefetching:
        A = np.random.rand(n_N, N, N)
        B = np.random.rand(n_M, M, M)
        C = np.zeros((n_N, n_M, N, N))
    else:
        A = np.random.rand(N, N, n_N)
        B = np.random.rand(M, M, n_M)
        C = np.zeros((N, N, n_N, n_M))
    return A, B, C

for prefetching in [True, False]:
    print ('with prefetching=', prefetching)
    A, B, C = get_data(prefetching=prefetching)
    print('Checking size of A =', A.shape, ' of B=', B.shape, ', and of C=', C.shape)

with prefetching= True
Checking size of A = (2, 1024, 1024)  of B= (3, 16, 16) , and of C= (2, 3, 1024, 1024)
with prefetching= False
Checking size of A = (1024, 1024, 2)  of B= (16, 16, 3) , and of C= (1024, 1024, 2, 3)


In [2]:
from numpy.fft  import fft2, ifft2
def np_fftconvolve(A, B):
    return np.real(ifft2(fft2(A)*fft2(B, s=A.shape)))
def test_numpy_fft(A, B, C, prefetching=False):
    if prefetching:
        for i_N in np.arange(A.shape[0]):
            for i_M in np.arange(B.shape[0]):
                C[i_N, i_M, :, :] = np_fftconvolve(A[i_N, :, :], B[i_M, :, :])
    else:
        for i_N in np.arange(A.shape[-1]):
            for i_M in np.arange(B.shape[-1]):
                C[:, :, i_N, i_M] = np_fftconvolve(A[:, :, i_N], B[:, :, i_M])

In [12]:
A, B, C = get_data(N, n_N, M, n_M)

In [25]:
%%timeit
test_numpy_fft(A, B, C)

692 ms ± 21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
A, B, C = get_data(N, n_N, M, n_M, prefetching=True)

In [28]:
%%timeit
test_numpy_fft(A, B, C, prefetching=True)

678 ms ± 9.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
from scipy.signal import fftconvolve
def test_scipy_fft(A, B, C, prefetching=False):
    if prefetching:
        for i_N in np.arange(A.shape[0]):
            for i_M in np.arange(B.shape[0]):
                C[i_N, i_M, :, :] = fftconvolve(A[i_N, :, :], B[i_M, :, :], mode='same')
    else:
        for i_N in np.arange(A.shape[-1]):
            for i_M in np.arange(B.shape[-1]):
                C[:, :, i_N, i_M] = fftconvolve(A[:, :, i_N], B[:, :, i_M], mode='same')

In [42]:
A, B, C = get_data(N, n_N, M, n_M)

In [43]:
%%timeit
test_scipy_fft(A, B, C)

342 ms ± 4.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
A, B, C = get_data(N, n_N, M, n_M, prefetching=True)

In [45]:
%%timeit
test_scipy_fft(A, B, C, prefetching=True)

319 ms ± 6.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [38]:
A.shape,B.shape,C.shape

((1024, 1024, 2), (16, 16, 3), (1024, 1024, 2, 3))

In [21]:
from numpy.fft  import fft2, ifft2
def test_numpy_fft_opt(A, B, prefetching=False):
    if prefetching:
        f_B = np.zeros((B.shape[0], A.shape[-2], A.shape[-1]), dtype=np.complex128)
        for i_M in np.arange(B.shape[0]):
            f_B[i_M, :, :] = fft2(B[i_M, :, :], s=A.shape[-2:])
        
        for i_N in np.arange(A.shape[0]):
            f_A = fft2(A[i_N, :, :])
            for i_M in np.arange(B.shape[0]):
                C[i_N, i_M, :, :] = np.real(ifft2(f_A*f_B[i_M, :, :]))
    else:
        f_B = np.zeros((A.shape[0], A.shape[1], B.shape[-1]), dtype=np.complex128)
        for i_M in np.arange(B.shape[-1]):
            f_B[:, :, i_M] = fft2(B[:, :, i_M], s=A.shape[:2])
        
        for i_N in np.arange(A.shape[-1]):
            f_A = fft2(A[:, :, i_N])
            for i_M in np.arange(B.shape[-1]):
                C[:, :, i_N, i_M] = np.real(ifft2(f_A*f_B[:, :, i_M]))

In [31]:
A, B, C = get_data(N, n_N, M, n_M, prefetching=False)

In [32]:
%%timeit
test_numpy_fft_opt(A, B, prefetching=False)

543 ms ± 5.96 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
import sys
from scipy import signal
from scipy import linalg
import numpy as np
x = [[1,0 ],
     [0,-1 ]]
x = np.array(x)
y = [[4,5],
     [3,2]]
#[2,3]
#[5,4]
y = np.array(y)
print("conv:\n" ,  signal.convolve2d(x , y , 'full'))
s1 = np.array(x.shape)
s2 = np.array(y.shape)
size = s1 + s2 - 1
fsize = 2 ** np.ceil(np.log2(size)).astype(int)
fslice = tuple([slice(0, int(sz)) for sz in size])
new_x = np.fft.fft2(x , fsize)
new_y = np.fft.fft2(y , fsize)
result = np.fft.ifft2(new_x*new_y)[fslice].copy()
print("\nfft for my method:\n" , np.array(result.real , np.int32))
print("\nfft:\n" , np.array(signal.fftconvolve(x ,y) , np.int32))

conv:
 [[ 4  5  0]
 [ 3 -2 -5]
 [ 0 -3 -2]]

fft for my method:
 [[ 4  5  0]
 [ 3 -2 -5]
 [ 0 -3 -2]]

fft:
 [[ 4  5  0]
 [ 3 -2 -5]
 [ 0 -3 -2]]


In [3]:
x=func(1,2,3,4)

In [4]:
x[0](*x[1:])

10

In [7]:
add(func(1,2,3,4))

(<function func_int at 0x7f4ea05859d8>, 1, 2, 3, 4)


In [5]:
x

(<function __main__.func_int(a, b, c, d)>, 1, 2, 3, 4)

In [2]:
import numpy as np

In [42]:
a=np.random.randint(0,9,(3,5,5))

In [49]:
b=(a>1)

In [50]:
b.sum()/b.size

0.7333333333333333

In [40]:
a.shape

(3, 5, 5)

In [1]:
import numpy as np

In [2]:
gg=np.random.random((8000,16,16,3))
np.random.shuffle(gg)

In [3]:
c=np.arange(64).reshape(16,4)

In [60]:
%timeit gg.shape[0]

95 ns ± 1.89 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [61]:
%timeit len(gg)

69.8 ns ± 1.6 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [4]:
# %%timeit
mn=gg.mean(axis=0)
st=gg.var(axis=0)

In [5]:
gg.shape,mn.shape,st.shape

((8000, 16, 16, 3), (16, 16, 3), (16, 16, 3))

In [6]:
xcap=((gg-mn)/st)

In [53]:
%timeit st=np.sqrt(((gg-mn)**2).mean(axis=0,keepdims=True))
%timeit st=gg.std(axis=0,keepdims=True)

21.8 ms ± 380 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
21.6 ms ± 353 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
xcap.shape

(8000, 16, 16, 3)

In [14]:
%timeit gg*3

9.07 ms ± 204 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%timeit 3*gg

8.88 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%timeit var=((gg-mn)**2).mean(axis=0)

In [19]:
%timeit 2*gg.sum(axis=0)

3.48 ms ± 40.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [18]:
%timeit (2*gg).sum(axis=0)

12.6 ms ± 167 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
class test(Layer):
    def __init__(self,a):
        super().__init__()
        print(super().__str__())
        self.a=a

In [3]:
b=test('eeee')
c=test('fff')(b)

test<__main__.test object at 0x7ff22b796730>
test<__main__.test object at 0x7ff22b7963a0>


In [4]:
a=test('abcd')([b,c])

test<__main__.test object at 0x7ff22b796880>


In [3]:
from nnet.layers import Layer,conv2d

Seed: 128
Seed: 210


In [4]:
import cupy as cp
from scipy import ndimage

In [26]:
a = np.random.randint(0,10,(4,4,3)).astype(np.float32,copy=False)
k = np.array([[1,1,1],[1,1,0],[1,0,0]]).repeat(3).reshape(3,3,3).astype(np.float32,copy=False)
kf= np.flip(k)#k[:,::-1,::-1,:].transpose(3,1,2,0)[0]

In [6]:
a.shape

(4, 4, 3)

In [7]:
out=ndimage.correlate(a, k, mode='constant',origin=(1,1,1))
out.transpose(2,0,1)

array([[[ 0.,  0.,  7.,  4.],
        [ 0.,  7., 20., 11.],
        [ 7., 20., 33., 30.],
        [ 9., 21., 41., 38.]],

       [[ 0.,  0., 12., 11.],
        [ 0., 12., 36., 28.],
        [12., 36., 62., 60.],
        [13., 31., 66., 59.]],

       [[ 0.,  0., 18., 17.],
        [ 0., 18., 56., 43.],
        [18., 56., 95., 81.],
        [21., 48., 85., 85.]]], dtype=float32)

In [8]:
cp.tensordot

<function cupy.linalg.product.tensordot(a, b, axes=2)>

In [9]:
out.shape

(4, 4, 3)

In [10]:
c=conv2d(input_shape=(4,4,3),kernels=k,biases=0)

IndexError: tuple index out of range

In [12]:
kf.shape

(3, 3, 3)

In [13]:
c.forward(np.array([a])).squeeze()

NameError: name 'c' is not defined

In [15]:
from cupyx.scipy import ndimage as cndimage
ad=cp.asarray(a)
kfd=cp.asarray(kf)

In [16]:
cndimage.convolve(ad, kfd, mode='constant').squeeze()

array([[[12., 18., 11.],
        [36., 56., 36.],
        [28., 43., 32.],
        [26., 35., 23.]],

       [[36., 56., 36.],
        [62., 95., 62.],
        [60., 81., 51.],
        [42., 62., 38.]],

       [[31., 48., 27.],
        [66., 85., 44.],
        [59., 85., 47.],
        [36., 65., 39.]],

       [[35., 40., 19.],
        [51., 67., 36.],
        [41., 70., 43.],
        [23., 48., 29.]]], dtype=float32)

In [17]:
from scipy import signal

In [18]:
signal.oaconvolve(a,kf,mode='same').squeeze()

array([[[12.00000111, 18.00000181, 11.00000094],
        [35.99999928, 56.00000037, 35.99999932],
        [27.99999885, 43.00000032, 31.99999872],
        [25.99999863, 35.00000028, 23.00000017]],

       [[35.99999974, 56.00000073, 36.00000011],
        [61.99999892, 95.        , 61.99999862],
        [59.99999856, 80.99999949, 50.9999991 ],
        [41.99999981, 62.00000052, 37.99999995]],

       [[30.99999974, 48.00000056, 26.99999988],
        [65.99999811, 84.99999939, 43.99999979],
        [58.9999986 , 84.99999878, 46.99999936],
        [36.00000038, 64.99999973, 38.99999904]],

       [[34.99999974, 40.00000079, 19.00000088],
        [50.99999887, 66.99999947, 36.00000017],
        [40.99999902, 69.99999861, 42.99999893],
        [23.00000032, 47.99999983, 28.99999923]]])

In [19]:
signal.correlate(a,k,mode='same').squeeze()

array([[[12., 18., 11.],
        [36., 56., 36.],
        [28., 43., 32.],
        [26., 35., 23.]],

       [[36., 56., 36.],
        [62., 95., 62.],
        [60., 81., 51.],
        [42., 62., 38.]],

       [[31., 48., 27.],
        [66., 85., 44.],
        [59., 85., 47.],
        [36., 65., 39.]],

       [[35., 40., 19.],
        [51., 67., 36.],
        [41., 70., 43.],
        [23., 48., 29.]]])